In [1]:
import pandas as pd
import numpy as np
import gc
import os
import datetime
import json

### Settings

In [2]:
with open('settings.json', 'r') as j:
    json_data = json.load(j)

project_path = json_data["project_path"]

data_path = os.path.join(project_path, "data")
python_path = os.path.join(project_path, "python")


data_chunks = os.path.join(data_path,"data_chunks")
output_data_path = os.path.join(data_path, "output")

In [3]:
upper_time_limit = datetime.datetime(2016, 5, 1, 0, 0)

### We read data for selected cells and transform it (pivot)

In [4]:
file_list = []
for (dirpath, dirnames, filenames) in os.walk(data_chunks):
        file_list.extend(filenames)
print(file_list)

['2011.csv', '2012.csv', '2013.csv', '2014.csv', '2015.csv', '2016.csv']


In [5]:
dts = []
for f in file_list:
    print(os.path.join(data_chunks, f))
    tmp = pd.read_csv(os.path.join(data_chunks, f), header = 0, index_col=0, parse_dates = ['Time'])
    print ((tmp.shape[0])/102.0) #comparison with previous run
    print ((tmp.shape[0])/24.0/102.0) #comparison with previous run
    dts.append(tmp)
    del(tmp)
    gc.collect()

c:\Users\Anna\Desktop\AI_camp\YT\data\data_chunks\2011.csv
8759.0
364.9583333333333
c:\Users\Anna\Desktop\AI_camp\YT\data\data_chunks\2012.csv
8783.0
365.9583333333333
c:\Users\Anna\Desktop\AI_camp\YT\data\data_chunks\2013.csv
8759.0
364.9583333333333
c:\Users\Anna\Desktop\AI_camp\YT\data\data_chunks\2014.csv
8759.0
364.9583333333333
c:\Users\Anna\Desktop\AI_camp\YT\data\data_chunks\2015.csv
8760.0
365.0
c:\Users\Anna\Desktop\AI_camp\YT\data\data_chunks\2016.csv
4368.0
182.0


In [6]:
t = pd.concat(dts, ignore_index = True)
print(t.shape)
t.head()

(4915176, 3)


,Time,Square_id,Counts
0,2011-01-29 02:00:00,1075,17.0
1,2011-01-29 02:00:00,1076,34.0
2,2011-01-29 02:00:00,1077,19.0
3,2011-01-29 02:00:00,1125,59.0
4,2011-01-29 02:00:00,1126,152.0


In [7]:
del(dts)
gc.collect()

20

In [8]:
d = pd.pivot_table(t, values='Counts', index=['Time'],columns=['Square_id'], aggfunc=np.sum)

In [9]:
d.head()

Square_id,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
Time,,,,,,,,,,,,,,,,,,,,,
2011-01-01 00:00:00,33.0,68.0,23.0,39.0,156.0,261.0,287.0,354.0,371.0,128.0,...,12.0,0.0,4.0,20.0,20.0,11.0,1.0,47.0,1.0,19.0
2011-01-01 01:00:00,42.0,68.0,31.0,59.0,182.0,256.0,245.0,264.0,252.0,116.0,...,10.0,0.0,4.0,22.0,13.0,10.0,5.0,34.0,4.0,18.0
2011-01-01 02:00:00,40.0,59.0,18.0,62.0,170.0,225.0,228.0,255.0,235.0,99.0,...,14.0,0.0,4.0,1.0,1.0,0.0,2.0,11.0,2.0,0.0
2011-01-01 03:00:00,35.0,52.0,18.0,47.0,129.0,216.0,208.0,213.0,183.0,85.0,...,7.0,0.0,4.0,2.0,1.0,0.0,0.0,12.0,0.0,0.0
2011-01-01 04:00:00,17.0,29.0,9.0,31.0,83.0,149.0,185.0,173.0,142.0,63.0,...,13.0,0.0,3.0,1.0,2.0,0.0,0.0,4.0,1.0,0.0


In [10]:
d.shape

(48188, 102)

In [11]:
d.index

DatetimeIndex(['2011-01-01 00:00:00', '2011-01-01 01:00:00',
               '2011-01-01 02:00:00', '2011-01-01 03:00:00',
               '2011-01-01 04:00:00', '2011-01-01 05:00:00',
               '2011-01-01 06:00:00', '2011-01-01 07:00:00',
               '2011-01-01 08:00:00', '2011-01-01 09:00:00',
               ...
               '2016-06-30 14:00:00', '2016-06-30 15:00:00',
               '2016-06-30 16:00:00', '2016-06-30 17:00:00',
               '2016-06-30 18:00:00', '2016-06-30 19:00:00',
               '2016-06-30 20:00:00', '2016-06-30 21:00:00',
               '2016-06-30 22:00:00', '2016-06-30 23:00:00'],
              dtype='datetime64[ns]', name='Time', length=48188, freq=None)

### Seach for no data in timeseries

#### the whole hour is missed for all squares

In [12]:
tmp = pd.DataFrame({"Time": list(d.index)
                    ,"next_time": list(d.index)[1:] + [None]})
tmp['delta'] = list(map(lambda x: x.seconds/3600.0, tmp["next_time"] - tmp ["Time"]))
tmp.loc[tmp.delta > 1.0].head()

,Time,next_time,delta
1705,2011-03-13 01:00:00,2011-03-13 03:00:00,2.0
10440,2012-03-11 01:00:00,2012-03-11 03:00:00,2.0
19175,2013-03-10 01:00:00,2013-03-10 03:00:00,2.0
27910,2014-03-09 01:00:00,2014-03-09 03:00:00,2.0


In [13]:
tmp.loc[tmp.delta.isna()]

,Time,next_time,delta
48187,2016-06-30 23:00:00,NaT,NaN


#### Given square has no data

In [14]:
print(d.shape)
print(d.dropna().shape)

(48188, 102)
(48188, 102)


### Fill empty data with average of neighbours

In [15]:
lost_data = tmp.loc[tmp.delta > 1.0]
for start_t, end_t, dlt in zip(lost_data.Time.tolist(), lost_data.next_time.tolist(),lost_data.delta.tolist()):
    values_to_fill = 0.5*(d.loc[d.index == start_t].values + d.loc[d.index == end_t].values)
    for i in range(int(dlt)-1):
        new_dt = pd.DataFrame(data=values_to_fill, columns=d.columns
                              ,index=pd.DatetimeIndex([start_t + datetime.timedelta(seconds = 3600*(i+1))], name="Time"))
        #print(new_dt.head())
        d = d.append(new_dt)
        
        

In [16]:
d.sort_index(inplace = True)

In [17]:
tmp = pd.DataFrame({"Time": list(d.index)
                    ,"next_time": list(d.index)[1:] + [None]})
tmp['delta'] = list(map(lambda x: x.seconds/3600.0, tmp["next_time"] - tmp ["Time"]))
tmp.loc[tmp.delta > 1.0].head()

,Time,next_time,delta


In [18]:
d.shape

(48192, 102)

### Data to file

In [19]:
d.reset_index(inplace=True)
d.head()

Square_id,Time,1075,1076,1077,1125,1126,1127,1128,1129,1130,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
0,2011-01-01 00:00:00,33.0,68.0,23.0,39.0,156.0,261.0,287.0,354.0,371.0,...,12.0,0.0,4.0,20.0,20.0,11.0,1.0,47.0,1.0,19.0
1,2011-01-01 01:00:00,42.0,68.0,31.0,59.0,182.0,256.0,245.0,264.0,252.0,...,10.0,0.0,4.0,22.0,13.0,10.0,5.0,34.0,4.0,18.0
2,2011-01-01 02:00:00,40.0,59.0,18.0,62.0,170.0,225.0,228.0,255.0,235.0,...,14.0,0.0,4.0,1.0,1.0,0.0,2.0,11.0,2.0,0.0
3,2011-01-01 03:00:00,35.0,52.0,18.0,47.0,129.0,216.0,208.0,213.0,183.0,...,7.0,0.0,4.0,2.0,1.0,0.0,0.0,12.0,0.0,0.0
4,2011-01-01 04:00:00,17.0,29.0,9.0,31.0,83.0,149.0,185.0,173.0,142.0,...,13.0,0.0,3.0,1.0,2.0,0.0,0.0,4.0,1.0,0.0


In [20]:
d_melt = d.melt(id_vars=['Time'], var_name='Square_id', value_name='Counts')

In [21]:
d_melt.head()

,Time,Square_id,Counts
0,2011-01-01 00:00:00,1075,33.0
1,2011-01-01 01:00:00,1075,42.0
2,2011-01-01 02:00:00,1075,40.0
3,2011-01-01 03:00:00,1075,35.0
4,2011-01-01 04:00:00,1075,17.0


In [22]:
d.to_csv(os.path.join(output_data_path, "input_dataset_pivoted.csv"), header=True, index=False)

In [23]:
d_melt.to_csv(os.path.join(output_data_path, "input_dataset_melted.csv"), header=True, index=False)